In [25]:
import os, json
import numpy as np
import pandas as pd

In [26]:
from libcbmwrapper import LibCBMWrapper
import libcbmconfig
import cbm_defaults

In [27]:
dllpath = r'C:\dev\LibCBM\LibCBM\x64\Debug\LibCBM.dll'
dlldir = os.path.dirname(dllpath)
cwd = os.getcwd()
os.chdir(dlldir)
dll = LibCBMWrapper(dllpath)
os.chdir(cwd)

In [28]:

dbpath = 'C:\dev\cbm_defaults\cbm_defaults.db'
test_dir = r"C:\dev\LibCBM\LibCBMPyTest\tests\test1"
yield_age_class_size = 10
yield_table_path = os.path.join(test_dir, "yield.csv")
yield_table_header_row = True
transtions_path = os.path.join(test_dir, "transition_rules.json")

#create a single classifier/classifier value for the single growth curve
classifiers_config = libcbmconfig.classifier_config([
    libcbmconfig.classifier("growth_curve", [
        libcbmconfig.classifier_value("growth_curve1")
    ])
])


transitions_config = []
species_reference = cbm_defaults.load_species_reference(dbpath, "en-CA")

merch_volume_to_biomass_config = libcbmconfig.merch_volume_to_biomass_config(
    dbpath, [
        libcbmconfig.merch_volume_curve(
            classifier_set = ['growth_curve1'], 
            softwood_species_id = species_reference["Spruce"]["species_id"], 
            softwood_age_volume_pairs=[
                    (0, 0.0),
                    (10, 10.0),
                    (20, 15.0),
                    (30, 20.0),
                    (40, 25.0),
                    (50, 30.0),
                    (60, 35.0),
                    (70, 40.0),
                    (80, 45.0),
                    (90, 50.0),
                    (100, 55.0),
                    (110, 60.0),
                    (120, 65.0),
                    (130, 70.0),
                    (140, 75.0),
                    (150, 80.0),
                    (160, 85.625),
                    (170, 90.73529412),
                    (180, 95.84558824),
                    (190, 100.9558824)],
            hardwood_species_id=None,
            hardwood_age_volume_pairs=None)
    ]) 



In [29]:
config = libcbmconfig.initialize_config(
    dbpath = dbpath,
    merch_volume_to_biomass = merch_volume_to_biomass_config,
    classifiers = classifiers_config,
    transitions = transitions_config,
    save_path = "config.json")
config["classifier_values"]

[{'id': 1, 'classifier_id': 1, 'value': 'growth_curve1', 'description': ''}]

In [30]:

dll.Initialize(libcbmconfig.to_string(config))

In [21]:
op = dll.AllocateOp(1)

In [22]:
nstands = 1
age = np.array([0],dtype=np.int32)
classifiers = np.ones((1,1), dtype=np.int32)
spatial_units = np.array([3],dtype=np.int32)
pools = np.zeros((1,len(config["pools"])))
pools[:,0] = 1.0


In [9]:
classifiers

array([[1]])

In [10]:
dll.GetMerchVolumeGrowthOps(
    op, 
    classifiers=classifiers,
    pools=pools,
    ages=age,
    spatial_units=spatial_units,
    last_dist_type=None,
    time_since_last_dist=None,
    growth_multipliers=None)

In [11]:
dll.ComputePools([op], pools)

In [12]:
pools

array([[1.        , 0.12552828, 0.71713448, 1.76236578, 0.358572  ,
        0.21974434, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [13]:
op

0

In [14]:
config["pools"]

[{'name': 'Input', 'id': 0, 'index': 0},
 {'name': 'SoftwoodMerch', 'id': 1, 'index': 1},
 {'name': 'SoftwoodFoliage', 'id': 2, 'index': 2},
 {'name': 'SoftwoodOther', 'id': 3, 'index': 3},
 {'name': 'SoftwoodCoarseRoots', 'id': 4, 'index': 4},
 {'name': 'SoftwoodFineRoots', 'id': 5, 'index': 5},
 {'name': 'HardwoodMerch', 'id': 6, 'index': 6},
 {'name': 'HardwoodFoliage', 'id': 7, 'index': 7},
 {'name': 'HardwoodOther', 'id': 8, 'index': 8},
 {'name': 'HardwoodCoarseRoots', 'id': 9, 'index': 9},
 {'name': 'HardwoodFineRoots', 'id': 10, 'index': 10},
 {'name': 'AboveGroundVeryFastSoil', 'id': 11, 'index': 11},
 {'name': 'BelowGroundVeryFastSoil', 'id': 12, 'index': 12},
 {'name': 'AboveGroundFastSoil', 'id': 13, 'index': 13},
 {'name': 'BelowGroundFastSoil', 'id': 14, 'index': 14},
 {'name': 'MediumSoil', 'id': 15, 'index': 15},
 {'name': 'AboveGroundSlowSoil', 'id': 16, 'index': 16},
 {'name': 'BelowGroundSlowSoil', 'id': 17, 'index': 17},
 {'name': 'SoftwoodStemSnag', 'id': 18, 'inde